In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-07-11 22:53:07 - INFO - Notebook execution started.


In [3]:
%%time

%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.
CPU times: user 19.7 ms, sys: 8.12 ms, total: 27.8 ms
Wall time: 1.18 s


In [4]:

# === Standard Library Imports ===
import os
import sys
import json
import warnings
from datetime import datetime
from pathlib import Path

# Define the relative path to the 'src' directory (two levels up from current working directory)
src_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add 'src' directory to system path for module imports (e.g., utils)
if src_path not in sys.path:
    sys.path.append(src_path)

# === Third-Party Imports ===
import numpy as np
import pandas as pd
import mlflow
from typing import List

# Import transformers from huggingface
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

#Import components of notebook
from core.extract_text.arxiv_search import ArxivSearcher
from core.generator.script_generator import ScriptGenerator
from core.analyzer.scientific_paper_analyzer import ScientificPaperAnalyzer
from core.deploy.text_generation_service import TextGenerationService

#import langchain libraries
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEndpoint
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

# === Project-Specific Imports (from src.utils) ===
from src.utils import (
    load_config_and_secrets,
    configure_proxy,
    initialize_llm,
    configure_hf_cache
)

## Model Service Registration

In this section, we implement the **Model Service**, a REST API responsible for serving the language model. The API is automatically documented using Swagger (via FastAPI), enabling interactive testing and clear documentation of the endpoints.

## Text Generation Service

This section demonstrates how to use our TextGenerationService from the src/service directory. This approach improves code organization by separating the service implementation from the notebook, making it easier to maintain and update.

In [5]:
%%time

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# In case you just want to run this cell without the rest of the notebook 
# (you still need to install the requirements and run the import block), run the following block:
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"
MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
config, secrets = load_config_and_secrets(CONFIG_PATH, SECRETS_PATH)

# Import the TextGenerationService class
from core.deploy.text_generation_service import TextGenerationService

mlflow.set_tracking_uri('/phoenix/mlflow')
# Set up the MLflow experiment
mlflow.set_experiment("Text-Generation-service")

# Define path to the model

# Check if the model file exists
if not os.path.exists(MODEL_PATH):
    print(f"Warning: Model file not found at {MODEL_PATH}. You may need to update the path.")

# Define demo folder path
DEMO_FOLDER = "../demo"

#Only logs the model path in the case where it is local
if config["model_source"] == "local":
    model_path = MODEL_PATH
else:
    model_path = None


# Use the TextGenerationService's log_model method to register the model in MLflow
with mlflow.start_run(run_name="Script-Generation") as run:
    # Log and register the model using the service's classmethod
    TextGenerationService.log_model(
        llm_artifact = MODEL_PATH,
        config_yaml  = CONFIG_PATH,
        secrets_yaml = SECRETS_PATH
    )
    
    # Register the model in MLflow Model Registry
    model_uri = f"runs:/{run.info.run_id}/script_generation_model"
    mlflow.register_model(model_uri=model_uri, name="Script-Generation-Service")
    print(f"Model registered successfully with run ID: {run.info.run_id}")

Registered model 'Script-Generation-Service' already exists. Creating a new version of this model...


Model registered successfully with run ID: 420d501ab6eb460ba26e0cb74673bb24
CPU times: user 760 ms, sys: 24.4 s, total: 25.2 s
Wall time: 2min 14s


Created version '16' of model 'Script-Generation-Service'.


In [6]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-07-11 22:55:27 - INFO - ⏱️ Total execution time: 2m 19.95s
2025-07-11 22:55:27 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).